In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
ensemble_csvs = [
    './mask2former_beit_adapter_large_640_27ecpoh_SH_7866.csv',
    './train_full_upernet_beit_adapter_large_640_160k_ade20k__schdule_difflr_no_lossweight_2_epoch_5_HH_7851.csv',
    './pseudo_upernet_beit_adapter_large_640_160k_ade20k_diff_4__epoch_5_HH_7836.csv',
    './mask2former_beit_adapter_large_640_finetuning SOTA model_finetuning_SH_7791.csv',
    './upernet_convnext_xlarge_fp16_epoch35_SH_7166.csv'
]

use_weights = False
weights = [1, 1, 1, 1, 1]  #weights 리스트의 길이는 ensemble_csvs 리스트의 길이와 같아야 함

dfs = [pd.read_csv(csv, index_col=False) for csv in ensemble_csvs]

In [ ]:
for i in range(len(dfs)-1):
    assert len(dfs[i]) == len(dfs[i+1]), '포맷이 다른 csv가 {} 번째에 존재'.format(i)


def ensemble(dfs, use_weights, *args):
    if use_weights:
        weights = args[0]
        loudness = np.argsort(np.array(weights))[::-1]
        print(weights)
        print(loudness)
    else:
        loudness = [1 for _ in range(len(dfs))]
        
    new_col = []
    
    for idx in tqdm(range(len(dfs[0]))):
        preds = []
        all_preds = []
        for i, df in enumerate(dfs):
            pred = df.iloc[idx, 1]
            pred = np.array(list(map(int, pred.split(" "))), dtype = int).reshape((256,256,1))
            all_preds.append(pred)
            if use_weights:
                for _ in range(weights[i]):
                    preds.append(pred)
            else:
                preds.append(pred)
                
        concat_pred = preds[0]
        for p in preds[1:]:
            concat_pred = np.concatenate([concat_pred, p], axis = -1)
        
        new_pred = np.zeros(shape=(256,256), dtype=int)
        for i in range(256):
            for j in range(256):
                count = np.bincount(concat_pred[i,j,:])
                maxcount = max(count)
                winners = np.argwhere(count == maxcount)
                if len(winners) > 1:
                    for l in loudness:
                        if all_preds[l][i, j] in winners:
                            winner = all_preds[l][i, j]
                            break
                else:
                    winner = winners[0]
                new_pred[i, j] = winner
                    
        new_col.append(' '.join(list(map(str, new_pred.flatten().tolist()))))
        
    
    return new_col

nc = ensemble(dfs, False)
        

In [ ]:
new_df = dfs[0].copy(deep=True)
new_df = new_df.drop(columns='PredictionString')
new_df['PredictionString'] = pd.Series(nc)

In [ ]:
new_df.describe()

In [ ]:
new_df.to_csv('./ensembled.csv', index=None)